In [ ]:
# [원어민vs사람 음성 유사도 평가 항목]
# 피치: 음성의 높낮이(억양, 리듬) 평가
        # 의미 전달(감정 표현/강조)
# 포먼트: 모음 발음의 정확도(각 모음은 고유한 포먼트 주파수 패턴을 가짐)
# MFCC: 모음 뿐만 아니라 모든 음소의 발음의 명확성/구별성(발음의 청각적 유사성 측정; 사람의 청각 특성에 맞춰진 Mel 주파수 스케일 사용)
# 타이밍: 발화 속도 평가

In [ ]:
# [유사도 평가시, 코사인 vs 유클리드 거리]
# 코사인 유사도와 유클리드 거리 중 코사인 유사도를 사용하는 것이 발음 비교에서는 더 적합할 수 있음
# 코사인 유사도는 두 벡터 간의 방향성(패턴의 유사성)을 평가: 크기보다 형태(발음 패턴의 유사성)를 평가할 때 유리함
# 특히 피치나 포먼트, MFCC는 주파수나 스펙트럼 패턴을 다루므로 코사인 유사도를 활용하는 것이 일반적

![발음 유사도 채점기준표](./스크린샷%202024-11-12%20124814.png)

In [5]:
import whisper
import parselmouth
import librosa
import numpy as np
from scipy.spatial.distance import cosine
from praatio import textgrid

In [1]:
# 음성 파일 및 MFA 결과 TextGrid 파일 경로 설정
audio_path_1 = "./audio/example_tts.wav"  # 원어민 음성
audio_path_2 = "./audio/user_audio.wav"  # 사용자 음성

In [5]:
# TextGrid 파일 생성
mfa align ./audio ./text ./english.dict ./english ./textgrid

SyntaxError: invalid syntax (2496835917.py, line 2)

In [3]:
# TextGrid 파일 경로 설정 (MFA 실행 후 생성된 파일 경로)
textgrid_path_1 = "./textgrid/example_tts.TextGrid"
textgrid_path_2 = "./textgrid/user_audio.TextGrid"

In [8]:
# Whisper 모델 불러오기
whisper_model = whisper.load_model("small")

c:\Users\user\anaconda3\envs\epa\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [ ]:
# 음성을 텍스트로 변환
def transcribe_audio(audio_path):
    result = whisper_model.transcribe(audio_path)
    print("음성 텍스트 변환 결과:", result["text"])
    return result["text"]

# 음성 텍스트 변환
text1 = transcribe_audio(audio_path_1)

In [ ]:
# TextGrid 파일 불러오기
tg_1 = tgio.openTextgrid(textgrid_path_1, includeEmptyIntervals=False)
tg_2 = tgio.openTextgrid(textgrid_path_2, includeEmptyIntervals=False)

In [ ]:
# 피치, 포먼트, MFCC 계산 함수
def analyze_features(audio_path, start, end):
    sound = parselmouth.Sound(audio_path)
    segment = sound.extract_part(from_time=start, to_time=end)
    
    # 피치
    pitch = segment.to_pitch().selected_array['frequency']
    
    # 포먼트
    formant = segment.to_formant_burg()
    formant_values = [formant.get_value_at_time(f, (start + end) / 2) for f in range(1, 4)]  # 포먼트 F1, F2, F3
    
    # MFCC
    y_segment, _ = librosa.load(audio_path, sr=sr1, offset=start, duration=(end - start))
    mfcc = librosa.feature.mfcc(y=y_segment, sr=sr1, n_mfcc=13)
    
    return pitch, formant_values, mfcc

In [ ]:
# 코사인 유사도 계산 함수
def cosine_similarity(vec1, vec2):
    if len(vec1) != len(vec2):
        return None
    return 1 - cosine(vec1, vec2)

In [ ]:
# 단어별 및 음소별 비교 함수
def compare_features(tg1, tg2, word_tier="words", phone_tier="phones"):
    word_intervals_1 = tg1.tierDict[word_tier].entryList
    word_intervals_2 = tg2.tierDict[word_tier].entryList
    phone_intervals_1 = tg1.tierDict[phone_tier].entryList
    phone_intervals_2 = tg2.tierDict[phone_tier].entryList
    
    for word_index, (word1, word2) in enumerate(zip(word_intervals_1, word_intervals_2)):
        word_label1, word_start1, word_end1 = word1.label, word1.start, word1.end
        word_label2, word_start2, word_end2 = word2.label, word2.start, word2.end
        
        # 동일한 단어인지 확인
        if word_label1 == word_label2:
            print(f"\n단어 '{word_label1}' (구간 {word_index + 1}) 비교:")
            
            # 단어 구간 내 음소별 비교
            for i, (phone1, phone2) in enumerate(zip(phone_intervals_1, phone_intervals_2)):
                # 단어 구간 내 음소만 필터링
                if phone1.start >= word_start1 and phone1.end <= word_end1 and \
                   phone2.start >= word_start2 and phone2.end <= word_end2:
                    
                    # 음소 정보
                    phone_label1, start1, end1 = phone1.label, phone1.start, phone1.end
                    phone_label2, start2, end2 = phone2.label, phone2.start, phone2.end
                    
                    # 동일한 음소인지 확인
                    if phone_label1 == phone_label2:
                        print(f"  음소 '{phone_label1}' (단어 '{word_label1}' 내 음소 {i + 1}) 비교:")
                        
                        # 각 음성의 피치, 포먼트, MFCC 계산
                        pitch1, formant1, mfcc1 = analyze_features(audio_path_1, start1, end1)
                        pitch2, formant2, mfcc2 = analyze_features(audio_path_2, start2, end2)
                        
                        # 코사인 유사도 계산 및 출력
                        pitch_similarity = cosine_similarity(pitch1, pitch2)
                        formant_similarity = cosine_similarity(formant1, formant2)
                        mfcc_similarity = cosine_similarity(np.mean(mfcc1, axis=1), np.mean(mfcc2, axis=1))
                        
                        # 타이밍 차이
                        timing_difference = (end2 - start2) - (end1 - start1)
                        
                        # 결과 출력
                        print(f"    피치 유사도: {pitch_similarity:.2f}")
                        print(f"    포먼트 유사도: {formant_similarity:.2f}")
                        print(f"    MFCC 유사도: {mfcc_similarity:.2f}")
                        print(f"    타이밍 차이: {timing_difference:.2f} 초")
                    else:
                        print(f"  Warning: 음소가 일치하지 않습니다 ('{phone_label1}' vs '{phone_label2}')")
        else:
            print(f"Warning: 단어가 일치하지 않습니다 ('{word_label1}' vs '{word_label2}')")

In [ ]:
# 단어 및 음소별 비교 실행
compare_features(tg_1, tg_2)